### Aufbau der Prolog Programme in Bash :D
```
cat <<EOF > filename.ecl
r:- [filename].
% Wissen
EOF

eclipse -f filename.ecl <<EOF
% Anfrage
EOF
```

In [1]:
# Anfrage: Welche Tiere gibt es alle in unserem System?

# Wir definieren die Relationen und Fakten:
cat <<EOF > saeugetier.ecl
r:- [saeugetier].
saeugetier(X) :- pflanzenfresser(X).
tier(X) :- saeugetier(X).
pflanzenfresser(jumbo).
EOF

# Die Datei mit dem 'Wissen' wird geladen
# und dann können wir anfragen an Prolog stellen:
eclipse -f saeugetier.ecl <<EOF
?-tier(X).
EOF

# Aktuell findet Prolog allerdings nur 1 Lösung :/

ECLiPSe Constraint Logic Programming System [kernel]
Kernel and basic libraries copyright Cisco Systems, Inc.
and subject to the Cisco-style Mozilla Public Licence 1.1
(see legal/cmpl.txt or http://eclipseclp.org/licence)
Source available at www.sourceforge.org/projects/eclipse-clp
GMP library copyright Free Software Foundation, see legal/lgpl.txt
For other libraries see their individual copyright notices
Version 6.1 #164 (x86_64_macosx), Mon Jul 15 23:37 2013
[eclipse 1]: 
X = jumbo
Yes (0.00s cpu)
[eclipse 2]: 
bye


In [2]:
cat <<EOF > berlin.ecl
r:- [berlin].
regierungssitz(berlin) :- hauptstadt(berlin).
hauptstadt(berlin).
EOF

eclipse -f berlin.ecl <<EOF
?-regierungssitz(berlin).
?-regierungssitz(X).
EOF

ECLiPSe Constraint Logic Programming System [kernel]
Kernel and basic libraries copyright Cisco Systems, Inc.
and subject to the Cisco-style Mozilla Public Licence 1.1
(see legal/cmpl.txt or http://eclipseclp.org/licence)
Source available at www.sourceforge.org/projects/eclipse-clp
GMP library copyright Free Software Foundation, see legal/lgpl.txt
For other libraries see their individual copyright notices
Version 6.1 #164 (x86_64_macosx), Mon Jul 15 23:37 2013
[eclipse 1]: 
Yes (0.00s cpu)
[eclipse 2]: 
X = berlin
Yes (0.00s cpu)
[eclipse 3]: 
bye


In [ ]:
cat <<EOF > einaus.ecl
r:- [einaus].
:- dynamic zustand/1.
zustand(ein).
einaus(S):-retract(zustand(ein)),S=aus, asserta(zustand(S)),!.
einaus(S):-retract()
EOF

eclipse -f einaus.ecl <<EOF

EOF